In [8]:

import base64
from jupyter_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "0925"
shelter = AnimalShelter(username, password, port=27017)

# class read method must support return of cursor object
df = pd.DataFrame.from_records(shelter.read({}), exclude=['_id'])

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML anchor/image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.A(html.Img(style={'height' : '10%', 'width' : '10%',
                            'display' : 'block', 'margin-left' : 'auto',
                            'margin-right' : 'auto', 'margin-top' : '10px'
                           }, src='data:image/png;base64,{}'.format(encoded_image.decode())), href='www.snhu.edu'),
    html.Center(html.B(html.H1('Fabian Rodriguez CS-340 Dashboard'))),
    html.Hr(),
    html.Div(

        # Add radio buttons for filtering, Water Rescue, Mountain and Wilderness Rescue, Disaster Rescue or Individual
        # Tracker, Reset
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
                {'label': 'All', 'value': 'All'}
            ],
            value='All',
            labelStyle={'display': 'inline-block'}
        ),
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
       editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type, **kwargs):
### FIX ME Add code to filter interactive data table with MongoDB queries
    # water_rescue = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland', 'Flat Coat Retriever']
    # mountain_rescue = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
    # disaster_rescue = ['Dobberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
    new_df = pd.DataFrame()
    if filter_type== 'All':
        new_df = pd.DataFrame.from_records(shelter.read({}))
    elif filter_type == 'WTR':
        new_df = pd.DataFrame(list(shelter.read({"breed" : {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}})))
    elif filter_type == 'MWR':
        new_df = pd.DataFrame(list(shelter.read({"breed" : {"$in": ["German Shephard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}})))
    elif filter_type == 'DIT':
        new_df = pd.DataFrame(list(shelter.read({"breed" : {"$in": ["Doberman Pinscher", "German Shephard", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 300, "$gte": 20}})))

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in new_df.columns]

    data=new_df.to_dict('records')


    return data, columns

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns, **kwargs):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData, **kwargs):
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    return [
       dcc.Graph(
           figure = px.pie(
               data_frame = dff,
               names = 'breed',
               title = 'Breed of Dogs'
           )
       )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData, **kwargs   ):

    dff = pd.DataFrame.from_dict(viewData)

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10,
               children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[30.75, -97.48], children=[
                    dl.Tooltip(dff.iloc[0, 4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[0, 9])
                    ])
                ])
            ]),
    ]


if __name__ == '__main__':
    app.run_server(debug=True)

Connected to MongoDB
Dash app running on http://127.0.0.1:8050/
